## Deliverable 2. Create a Customer Travel Destinations Map.

In [47]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [48]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

City_ID       City Country                 Date    Lat    Lng  Max Temp  \
0        0     Castro      CL  2019-08-26 17:25:49 -42.48 -73.76     48.20   
1        1       Lebu      ET  2019-08-26 17:25:49   8.96  38.73     58.69   
2        2      Sitka      US  2019-08-26 17:25:49  37.17 -99.65     90.00   
3        3     Torbay      CA  2019-08-26 17:25:49  47.66 -52.73     75.00   
4        4  Grandview      US  2019-08-26 17:25:50  38.89 -94.53     69.80   

   Humidity  Cloudiness  Wind Speed   Current Description  \
0        61          40       14.99      scattered clouds   
1        83          72        1.45            light rain   
2        46           6       21.00             clear sky   
3        43          20       10.29            few clouds   
4        93          40        4.70  heavy intensity rain   

   Rain inches (last 3 hrs)  Snow inches (last 3 hrs)  
0                     0.000                         0  
1                     2.187                         0  
2                     0.000                         0  
3                     0.000                         0  
4                     0.000                         0

In [49]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# min_temp = input("Minimum temperature desired for vacation")
# Max_temp = input("Maximum temperature desired for vacation")

min_temp = 40
max_temp = 100

In [50]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
prefer_city_df = city_data_df.loc[(city_data_df["Max Temp"]<=max_temp) & (city_data_df["Max Temp"]>=min_temp)]
prefer_city_df

City_ID         City Country                 Date    Lat     Lng  \
0          0       Castro      CL  2019-08-26 17:25:49 -42.48  -73.76   
1          1         Lebu      ET  2019-08-26 17:25:49   8.96   38.73   
2          2        Sitka      US  2019-08-26 17:25:49  37.17  -99.65   
3          3       Torbay      CA  2019-08-26 17:25:49  47.66  -52.73   
4          4    Grandview      US  2019-08-26 17:25:50  38.89  -94.53   
..       ...          ...     ...                  ...    ...     ...   
560      560      Dauphin      CA  2019-08-26 17:28:27  51.15 -100.04   
561      561   Deep River      CA  2019-08-26 17:28:27  46.10  -77.49   
562      562  Port-Gentil      GA  2019-08-26 17:28:27  -0.72    8.78   
563      563       Eureka      US  2019-08-26 17:28:27  40.80 -124.16   
564      564         Moen      BE  2019-08-26 17:28:28  50.77    3.40   

     Max Temp  Humidity  Cloudiness  Wind Speed   Current Description  \
0       48.20        61          40       14.99      scattered clouds   
1       58.69        83          72        1.45            light rain   
2       90.00        46           6       21.00             clear sky   
3       75.00        43          20       10.29            few clouds   
4       69.80        93          40        4.70  heavy intensity rain   
..        ...       ...         ...         ...                   ...   
560     68.00        72          90        9.17       overcast clouds   
561     78.01        60          20        8.03            few clouds   
562     78.80        83          75       10.29         broken clouds   
563     73.00        68           1        3.36                  haze   
564     90.00        40           0        5.82             clear sky   

     Rain inches (last 3 hrs)  Snow inches (last 3 hrs)  
0                       0.000                         0  
1                       2.187                         0  
2                       0.000                         0  
3                       0.000                         0  
4                       0.000                         0  
..                        ...                       ...  
560                     0.000                         0  
561                     0.000                         0  
562                     0.000                         0  
563                     0.000                         0  
564                     0.000                         0  

[550 rows x 13 columns]

In [5]:
# 4a. Determine if there are any empty rows.
prefer_city_df.count()

City_ID                     550
City                        550
Country                     545
Date                        550
Lat                         550
Lng                         550
Max Temp                    550
Humidity                    550
Cloudiness                  550
Wind Speed                  550
Current Description         550
Rain inches (last 3 hrs)    550
Snow inches (last 3 hrs)    550
dtype: int64

In [51]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = prefer_city_df.dropna()

In [52]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

City Country  Max Temp   Current Description    Lat  \
0                 Castro      CL     48.20      scattered clouds -42.48   
1                   Lebu      ET     58.69            light rain   8.96   
2                  Sitka      US     90.00             clear sky  37.17   
3                 Torbay      CA     75.00            few clouds  47.66   
4              Grandview      US     69.80  heavy intensity rain  38.89   
6               Cidreira      BR     72.01      scattered clouds -30.17   
7                   Hilo      US     75.20  heavy intensity rain  19.71   
8            Great Falls      US     63.00             clear sky  47.50   
9                   Lata      IN     41.77            light rain  30.78   
10  Imeni Stepana Razina      RU     54.91       overcast clouds  54.89   

       Lng Hotel Name  
0   -73.76             
1    38.73             
2   -99.65             
3   -52.73             
4   -94.53             
6   -50.22             
7  -155.08             
8  -111.29             
9    78.62             
10   44.30

In [53]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(url, params=params).json()
    print(hotels)
# 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_dl.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
             print("not found")
        

{'html_attributions': [], 'next_page_token': 'ATtYBwLEaacEdw4vQ7b-JLTdQNn_Je_PTmZHOOa1GRaw9HsigmKFF32wUJFakEmgMzolISG9QKbBJkYZLw5Ffgp1xTgMaP8atlRE1seMoqSHiK3lTnMdwXYE09wr5VcNRaLMSQfjk3uhyjTwlR_vqIN9e03xxhT0JoWP2QS-sYHIUsjo459G9DTRJToYEnRnNSkufH5gXYAkQhPCyeYvJeoBV0TzIIQzjYo95Am6eVD4Jo9_uVmbPe4fRuJ2k4JOCZ6RL1mSMNc-uRPw8UkX2sZ6BRbKJa02eYgJNklk6M1kZEtZKKq7sOufIByxXeNaKAAee7I7ef47821fhh_FVem4mjQah_tkAlswplt61rJivG2Is20QTzz6xY7p6m7zZ4QEvTXbjRx1jUo5wpieAmcHBjBttEZqyPs5A3LewnZLXVA_-vOsadTP2BOw', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -42.4833677, 'lng': -73.7635708}, 'viewport': {'northeast': {'lat': -42.4820168697085, 'lng': -73.7621602697085}, 'southwest': {'lat': -42.48471483029149, 'lng': -73.76485823029151}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Hotel Esmeralda', 'opening_hours': {'open_now': True}, 'photos': [{'height': 632, 'html_attributions': ['<a href="https://maps.google.com/maps/cont

{'html_attributions': [], 'next_page_token': 'ATtYBwIiXMzu6SuVJCjlwkrLgoGZ5QofAhcdABs8nKD4oTcp0sOqgjs0uqhaFRIpr5rRkxdxdIrEWQAzh6P699Dx1jdjDa_xC3_TFcFMLOl9XcUTrRo3R9Uo-54TyleGl_IKPAlJ2qSyq8VBu80XmsAL1gzncGCHtQKuzx1OvzlvkilLNSOlmX_KI9KA6f7z9FEz4eZxeLWFK6xU8hDT8jqBxi3K2_gauxGqUNs610v3nZPT_Sxn42Kgqsz0xCUgi0P5W-c85PkkVnM8cnQxM5pZSeDv96g3Ee3gRYdjXbg46ZExtaja0ZjlQbQ7R1YjQLbEPHvrKWyzdtHNboeltTe8L073139hywAQbuUrLCiqpm8q36D8c2nR0ZW2JYD-ewM5GEgSU07rbHxfuWNRph1I8eVcEOX5XU6fpyMFbIuM-c3CtY8qUsYZrLh8', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 8.979045599999997, 'lng': 38.7490492}, 'viewport': {'northeast': {'lat': 8.9803290302915, 'lng': 38.7503936802915}, 'southwest': {'lat': 8.977631069708496, 'lng': 38.7476957197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Dah-Lak Guest House', 'photos': [{'height': 3264, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/116533553529664293736">Paulos

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 47.6595498, 'lng': -52.7202597}, 'viewport': {'northeast': {'lat': 47.6610302802915, 'lng': -52.71885751970851}, 'southwest': {'lat': 47.6583323197085, 'lng': -52.72155548029151}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'See The Sea', 'photos': [{'height': 266, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/111823487149586838390">See The Sea</a>'], 'photo_reference': 'ATtYBwI4I5eBzZyMTqcnRYS0dZ01JtTMoc9LaxPq5eEwCJ129isI-HagfFipjNe8v5AF-4USeZLwf8_LJ8MYbHX8b_igAnkbDC2ip6aGXXrBQvRNK6ecRVPAa3qtc7DrQ85SDTrDK4jKcaslh-LPkXSFcqWW7a6xKbO7QAx9AY_803N37d6i', 'width': 400}], 'place_id': 'ChIJQ_LbuKSnDEsRzAGWMKJ9KJQ', 'plus_code': {'compound_code': 'M75H+RV Torbay, NL, Canada', 'global_code': '88V9M75H+RV'}, 'rating': 5, 'reference': 'ChIJQ_LbuKSnDEsRzAGWMKJ9KJQ', 'scope': 'GOOGLE', 'types': ['lodging', 'point_of_inte

{'html_attributions': [], 'next_page_token': 'ATtYBwINMDBl0rde5fdfeYhZHL-o6rGJiwss4CAXahjyLXDAXpUjwBvpeEY31nqz6onmeFKL1IE_evXMWENDEmXkK2In7LitgKTMI0ZRBRpjePQHeKmOICrdp7-WTlyxRnLaRGfBj3GLCs7YcIdV9aZefxhOitTUFMvQad33izbdqjVOG3x_yQxktlaASc2Xrn6AqyGy1kU5zg3IzXz6g6p7r4i5gZXmQuN5e8Dlox4qx1hq9nTv6xt-BsB294OcPAvhIDQ_Vsw0mx_D_YVohi8Eb8mqFzjp4qe7OijJA__ZL0TM2Dq6Oss3I89K8NqzR8G6k64bKw4oRypqZePeRnjbQp_LFBcwT6yazfFa63Jt6s8MkBVug2GaMmlx7ucMs9e3KQ5cEY6PcOAoEBZcK9gKWEAJa9CJ0l_RCKdjLZt9-E4StdtmaKYdS6gD', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -30.17788070000001, 'lng': -50.20350059999999}, 'viewport': {'northeast': {'lat': -30.17650586970851, 'lng': -50.20220251970849}, 'southwest': {'lat': -30.17920383029151, 'lng': -50.20490048029149}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Hotel Castelo', 'photos': [{'height': 1800, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/108481048178480058

{'html_attributions': [], 'next_page_token': 'ATtYBwJ67t6q9XKwyMvNQcQ6NrrMVRzOPyEq-Z48osRZO0-hNJZo2IUsm1XIVgK0fU5bDmENhay9D7fLyJaktXpXWzAj7bDRJ2C5lfFtMjcGPevMPbmZrc8-xZ-g8AgxIhPanVNYfyLNiPHiaYtAucy-xUQlTtYylHrYkwf5hn4mNrMey7-Yji3g3-o_bgI84_7CNfJ4oNZchX82-u9cGGUN4BLVJfp85iLknZZFH60A9x8INzp4G7ML3_Mv0EMjHYjjntsmOQ-AtQmc_odiI2qMrgpJm-EFxXsxaS-8jVu2c9vWm5_z3IzgfJo6vFxAv1l3x2rVk5koWfEV29gHovyvKgNHHBFW8mqjsxUqfWTDxPTZnKRKpa6ax0kpkGv1UDrYGnLUWs0D8fBp3wVY9qos54iKMR2QHE8kItICrji1kVpbFzpanw0HSFkl', 'results': [{'business_status': 'CLOSED_TEMPORARILY', 'geometry': {'location': {'lat': 19.723507, 'lng': -155.062949}, 'viewport': {'northeast': {'lat': 19.7249923802915, 'lng': -155.0613928697085}, 'southwest': {'lat': 19.7222944197085, 'lng': -155.0640908302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Hilo Seaside Hotel', 'permanently_closed': True, 'photos': [{'height': 1741, 'html_attributions': ['<a href="https://maps.google.com/maps/contri

...

{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
not found
{'html_attributions': [], 'next_page_token': 'ATtYBwKbaHmca5Q1mrJWkIkuRmjj40Pmvhlm6qzUzGasp9yZs7jggD5S5neN15TJPUifbc_eF0sOgCafxLQovvwxL0q1aG6pvi1Iw0lM11VzG-4-dUEg1fFeFFe-fivoq6vlZQml70jPBRR8FLepppKkZD4MZRqeE2bkNANaTA43V-Sq9KbURfnghrz9LFj2hwJrMvDOjO2vmOnshj6c0q3A9-5sC-4BIpZWf56QS0Yv0zxDi98UnO6MfVIyjYpQ6dc75RLn4OrrkJ-Ts66omXAC4gsMrzAx1rIcohDfuN9ta5E6ZMgTcpozvxyH-0aNwuTvYvSViV3GioabV5kHDINitw_Yny5mXOHKbhSnBAoxg2ZBZDWPRx_jc8cyQjAuK-GitZIpp5pfYPptJgv663J-_IaJmaaoVgwkNJt0IV2ILMi2zp2PJu_umuxE', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 49.1462081, 'lng': -102.9659891}, 'viewport': {'northeast': {'lat': 49.14783353029149, 'lng': -102.9646560197085}, 'southwest': {'lat': 49.14513556970849, 'lng': -102.9673539802915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Motel 6 Estevan, SK', 'opening_hours': {'open_now': True}, 'photos': [

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 45.63132530000001, 'lng': -73.9399775}, 'viewport': {'northeast': {'lat': 45.6325720802915, 'lng': -73.93876381970848}, 'southwest': {'lat': 45.6298741197085, 'lng': -73.94146178029149}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/camping-71.png', 'name': 'Camping Mirabel', 'photos': [{'height': 1305, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/104403271033660257933">Marianne Dubé</a>'], 'photo_reference': 'ATtYBwK7NDYRK2bXunTDnCLXT5c9mqs6bG0LfH_QFfpjF71p6zx490BjDyHav9Izv5wpjQ8aY4582XfUsX3TGBXV0e1DtHoLTVZnXF6LO-nXNo7HaHQwsHrV4et96eSq1QA4B6_wpBunsqp1Cad1xrTSOOkXKEiXNxLdlfkTTgAq44MAm0l5', 'width': 2160}], 'place_id': 'ChIJ8TQPAm8pyUwRRDI0YZZBM0g', 'plus_code': {'compound_code': 'J3J6+G2 Mirabel, QC, Canada', 'global_code': '87Q8J3J6+G2'}, 'rating': 4, 'reference': 'ChIJ8TQPAm8pyUwRRDI0YZZBM0g', 'scope': 'GOOGLE', 'types': ['campgrou

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 46.109825, 'lng': -77.522624}, 'viewport': {'northeast': {'lat': 46.1113006802915, 'lng': -77.5211728197085}, 'southwest': {'lat': 46.1086027197085, 'lng': -77.5238707802915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Deep River Motel', 'photos': [{'height': 3000, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/113756057221243524283">Deep River Motel</a>'], 'photo_reference': 'ATtYBwIV_aWyFG3lAyWBbLWl_bfwFG2y4tDLJ-FCWjdX4Lxdz7JRtlQS2teVvvw1yXeQxjjBqlULNplWaX6zu0K513BlSAKnueWbIPnTLs2y6CtJo-AVGGVrohx12P7T_m3oOLG4Oj1qgaFwLwKKsnCtqySvo6ZITsAqDFjqUQXLelUNdGk4', 'width': 4000}], 'place_id': 'ChIJq51g1K_f1kwRw0LsyvAAB40', 'plus_code': {'compound_code': '4F5G+WX Deep River, ON, Canada', 'global_code': '87R44F5G+WX'}, 'rating': 3.9, 'reference': 'ChIJq51g1K_f1kwRw0LsyvAAB40', 'scope': 'GOOGLE', 'types': ['lodging', 

{'html_attributions': [], 'next_page_token': 'ATtYBwJLkIgu37GMMbLZNSaRSX_FW1LGiyzTEhzIZsnloZzLaTjdtwA04u6NJ5H8znTS3YbBRK0BVPx6d-rm6oXahTiEZZmyoIRQpuRbiHITwgQeOjEWHcNfo637HEULkLv3ZEDjy5PPY1MT4vmtqA7jF5hCGyB4wZu700YYaJs8F01bHWQg6sDcSsbLYrgOisp1amo-oT8o2jA4xfixNc2HveNFuvGz8ETfeEeAqakxQYXAsf0_FzC8_7blCwYS7jo3SNmKG8xp4hAAk4KRsZ5hybGdKKBtNVi7hEY9NRzfksx0vlosSdqDF4AOoh-Xi9qbDTlXMJYVuAmDT5LiHn53gGtnrpFd_tOrQ4oBtpr_yjYCM--Oe6QMyy_Z0J_E-qKmS0U5Tk1cvXUHJv4aPEPTfEBGnYIXaxf1wb5H9W-9s5QSEF-Q-y3rCmnc', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 40.80010619999999, 'lng': -124.1653707}, 'viewport': {'northeast': {'lat': 40.80157193029149, 'lng': -124.1639348197085}, 'southwest': {'lat': 40.7988739697085, 'lng': -124.1666327802915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Eureka Inn Trademark By Wyndham', 'photos': [{'height': 1575, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/1134632680

{'html_attributions': [], 'next_page_token': 'ATtYBwKtA7MVjZ2wyScIxNqmNcG3UGWR1tYwFPkg59MWMUy5D8t_lwu1T-S6WerdMV7bubbLJMSBfV6pvkPna-Yu0tkhzUBHhVEFtQCgZse4rYFmCag7-AElO2IzO5auZj8jcZAu-bnwDDcqoULq9g-iySBL4VZG-YLyuwC6KhxdHKYbJuobbQ7b41HiaRkJhULwWnMsUpTu_uckWU9bvCVRw7DhXwxWpJBuYVOnD9ToxxYEaKJ6-BJlqoWQWrPDzySw_TPeMKQokyQd2rxoJBlOIMVDpdAnPUZElz7rH5zPye_7QaqoGMJnjI2KOm7wyeJ9ChB3TN47ZYmY8YNlsPTNs2n92T1sqIzLQd_7R0SoDQXdzXNr6JMilB-Pf44twTamAlDIS8dIrf1Ksuiq5s3sa-ZaofZT1cShXT92I2muNUBVWOrH4vCfj1uA', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 50.7714357, 'lng': 3.4546524}, 'viewport': {'northeast': {'lat': 50.77271383029149, 'lng': 3.456062180291501}, 'southwest': {'lat': 50.77001586970849, 'lng': 3.453364219708497}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': "'t Bed & de Tafel", 'photos': [{'height': 4160, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/113616792220918067095">Sun Shine</a

In [54]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [55]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
output_data_file = "test.csv"
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [56]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [58]:
# 11a. Add a marker layer for each city to the map. 
# 11a. Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
# 11b. Display the figure
fig
# 11b. Display the figure


Figure(layout=FigureLayout(height='420px'))